In [50]:
from dotenv import load_dotenv
load_dotenv()

True

In [51]:
from langchain.agents import create_agent
from langchain.messages import HumanMessage
from langchain.tools import tool
from tavily import TavilyClient
from langgraph.checkpoint.memory import InMemorySaver

In [52]:
import sys 
import asyncio

#Fix Windoiws issues in jupyter notebooks
if sys.platform == "win32":
    # 1. Use ProactorEventLoop for subprocess support
    if not isinstance(asyncio.get_event_loop_policy(),asyncio.WindowsProactorEventLoopPolicy):
        asyncio.set_event_loop_policy(asyncio.WindowsProactorEventLoopPolicy())
    
    # 2. Redirectstderr to avoid fileno() error when launching MCP servers
    if "ipykernel" in sys.modules:
        sys.stderr=sys.__stderr__

In [53]:
from langchain_mcp_adapters.client import MultiServerMCPClient

In [54]:
client=MultiServerMCPClient(
    {
        "travel_server":{
            "transport":"streamable_http",
            "url": "https://mcp.kiwi.com"
        }
    }
)

tools=await client.get_tools()

In [55]:
destination=input("Enter destination for your destination wedding")
current=input("Enter current location")

In [56]:
from typing import Dict,Any
tavily_client=TavilyClient()

@tool
def web_search(query:str) -> Dict[str,Any]:
    """Search the web for information"""
    return tavily_client.search(query)


In [57]:
flights_agent=create_agent(model="gpt-5-nano",tools=tools,system_prompt="you are a friendly flight booking agent. look up flight details at the user's request")
catering_agent=create_agent(model="gpt-5-nano",tools=[web_search],system_prompt="you are a friendly catering planning agent. you look for wedding catering services at location given by user")
photography_agent=create_agent(model="gpt-5-nano",tools=[web_search],system_prompt="you are a friendly photography planning agent. you look for wedding photography services at location given by user")
DJ_agent=create_agent(model="gpt-5-nano",tools=[web_search],system_prompt="you are a friendly DJ planning agent. you look for famous DJs at location given by user")
decorator_agent=create_agent(model="gpt-5-nano",tools=[web_search],system_prompt="you are a friendly wedding decoration planning agent. you look for wedding decorators services at location given by user")

In [58]:
import asyncio
@tool
async def calling_flights_agent(destination,current):
    """Call flights_agent to get flights from current location to destination"""
    response=await flights_agent.ainvoke({"messages":HumanMessage(content=f"Give me flights from {current} to {destination}")})
    return response["messages"][-1].content

In [59]:
@tool
async def calling_catering_agent(destination):
    """Call catering_agent to get catering services at destination"""
    response=await catering_agent.ainvoke({"messages":HumanMessage(content=f"Give me wedding catering services at {destination}")})
    return response["messages"][-1].content

In [60]:
@tool
async def calling_photography_agent(destination):
    """Call photography_agent to get photography services at destination"""
    response=await photography_agent.ainvoke({"messages":HumanMessage(content=f"Give me wedding photography services at {destination}")})
    return response["messages"][-1].content

In [61]:
@tool
async def calling_DJ_agent(destination):
    """Call DJ_agent to get DJ services at destination"""
    response=await DJ_agent.ainvoke({"messages":HumanMessage(content=f"Give me wedding DJ services at {destination}")})
    return response["messages"][-1].content

In [62]:
@tool
async def calling_decorator_agent(destination):
    """Call decorator_agent to get decorator services at destination"""
    response=await decorator_agent.ainvoke({"messages":HumanMessage(content=f"Give me wedding decorator services at {destination}")})
    return response["messages"][-1].content

In [63]:

wedding_planner_agent=create_agent(model="gpt-5-nano",tools=[calling_flights_agent,calling_catering_agent,calling_decorator_agent,calling_DJ_agent,calling_photography_agent],system_prompt=f"You are a friendly wedding planner agent. Plan for the coiple a dream destination wedding in {destination}. Use the tools you have to plan flights from their current location {current},plan catering,DJ,photography,decorator services. ",checkpointer=InMemorySaver(),)

In [64]:
config={"configurable":{"thread_id":"1"}}
question=HumanMessage(content="Plan my dream wedding")
response= await wedding_planner_agent.ainvoke({"messages":[question]},config,)

In [ ]:
print(response['messages'][-1].content)

Amazing—I’d love to map out a dream destination wedding for you in Switzerland. Here’s a structured plan covering flights from Bangalore and the core vendor categories (catering, décor, DJ, photography). I’ve pulled in solid Zurich-area options from the vendor lists I have, with quick notes and direct links so you can start reviewing.

Flights from Bangalore (BLR) to Zurich (ZRH)
- What I can do next: pull live flight options tailored to your dates, passenger count, cabin, and flexibility. I’ll compare price, total travel time, layovers, and baggage.
- Typical approach I recommend:
  - Look for 1-stop options via major hubs (DXB, AUH, DEL, MAA, FRA, MUC) or a multi-city route with a short layover.
  - Consider Swiss/ Lufthansa/ Air India/ Emirates partners for smoother connection times and easier group check-in.
  - If guests are coming for the wedding, we’ll plan a “welcome weekend” window with arrival 2–3 days before the ceremony.
  - Visa and travel: I can include guidance on Switze

: 